# Logistic Regression
classification 문제

In [1]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Traning Data

|x_data| = (6, 2)

|y_data| = (6, )

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis

We can use `torch.exp` to compute the hypothesis function conviniently.

In [6]:
W = torch.zeros((2, 1), requires_grad = True) # dim = 2
b = torch.zeros(1, requires_grad = True)

hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W)) + b))

print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


pytorch에서 sigmoid 함수 `torch.sigmoid()`를 제공하고 있기 때문에 굳이 수식으로 hypothesis를 적을 필요 없음.

In [7]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)

print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


We want to measure the difference between `hypothesis` and `y_train`

In [8]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


To compute the losses for the entire batch, we can simply input the entire vector.

In [ ]:
losses = -(y_train * torch.log(hypothesis) + (1-y_train) * torch.log(1-hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


Then, we just `.mean()` to take the mean of these individual losses.

In [ ]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


위의 긴 수식의 과정들을 `binary_cross_entropy`로 구할 수 있음.

In [9]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

# Whole Training Procedure

In [11]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = torch.optim.SGD((W, b), lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  # cost로 H(x) 계산
  optimizer.zero_grad() # 초기화를 하지 않으면 기존 grad에 더해지므로 꼭 초기화하기.
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch: {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch:    0/1000 Cost: 0.693147
Epoch:  100/1000 Cost: 0.134722
Epoch:  200/1000 Cost: 0.080643
Epoch:  300/1000 Cost: 0.057900
Epoch:  400/1000 Cost: 0.045300
Epoch:  500/1000 Cost: 0.037261
Epoch:  600/1000 Cost: 0.031672
Epoch:  700/1000 Cost: 0.027556
Epoch:  800/1000 Cost: 0.024394
Epoch:  900/1000 Cost: 0.021888
Epoch: 1000/1000 Cost: 0.019852


# Evaluation

원래는 `x_test`, 즉 test data set에 대해서 수행을 해줘야함.

In [13]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [15]:
prediction = hypothesis >= torch.FloatTensor([0.5]) # prediction type : byteTensor
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [16]:
# compare with y_train
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [21]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])
accuracy = torch.mean(correct_prediction.float())
print(accuracy)

tensor([[True],
        [True],
        [True],
        [True],
        [True]])
tensor(1.)


# Hyper Implementation with class

In [37]:
class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    # W와 b가 들어있는 linear layer
    # self.linear = {W, b}
    self.linear = nn.Linear(2, 1) # dim = 2이므로 nn.Linear(2, 1)이 맞음.
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    return self.sigmoid(self.linear(x))

In [38]:
model = BinaryClassifier()

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr = 1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  hypothesis = model(x_train)

  # cost 계산
  cost = F.binary_cross_entropy(hypothesis, y_train)

  # cost로 H(x) 계산
  optimizer.zero_grad() # 초기화를 하지 않으면 기존 grad에 더해지므로 꼭 초기화하기.
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 10 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5])
    correct_prediction = prediction.float() == y_train
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('Epoch: {:4d}/{} Cost: {:.6f} Accuracy: {:2.2f}%'.format(
        epoch, nb_epochs, cost.item(), accuracy*100
    ))

Epoch:    0/100 Cost: 0.590113 Accuracy: 50.00%
Epoch:   10/100 Cost: 0.442518 Accuracy: 66.67%
Epoch:   20/100 Cost: 0.412972 Accuracy: 83.33%
Epoch:   30/100 Cost: 0.346455 Accuracy: 83.33%
Epoch:   40/100 Cost: 0.292822 Accuracy: 83.33%
Epoch:   50/100 Cost: 0.244709 Accuracy: 100.00%
Epoch:   60/100 Cost: 0.201719 Accuracy: 100.00%
Epoch:   70/100 Cost: 0.168851 Accuracy: 100.00%
Epoch:   80/100 Cost: 0.150460 Accuracy: 100.00%
Epoch:   90/100 Cost: 0.139416 Accuracy: 100.00%
Epoch:  100/100 Cost: 0.130253 Accuracy: 100.00%
